Main goal:
 - summary the indonesia text using machine learning approach

Dataset:
 - twitter
 - news
 - blogger

Steps:
 1. pre-processing:
    - colletion data and cleansing the data
    - reduce the problem dimensionality (stop words, stemming, features selection, etc.)
 2. dataset modeling: 
    - deciding how to construct training datadatset. 
 3. Analysis: 
    - summary indonesian text using one or more algorithm. many options: Reinforcement Algorithm (Jiwanggi & Adriani, 2016), k-nn, SVM, Naive Bayes, Neural network


Referencess:
 Summary indonesian text
  - Meganingrum Arista Jiwanggi, Mirna Adriani. 2016, "Topic Summarization of Microblog Document in Bahasa Indonesia using the Phrase Reinforcement Algorithm"

 Pre-processing models (OPTIONS)
  - Mirna Adriani, Jelita Asian, Bobby Nazief, Seyed MM Tahaghoghi, Hugh E Williams. 2007, "Stemming Indonesian: A confix-stripping approach" 
  - Mirna Adriani. 2000, "Using statistical term similarity for sense disambiguation in cross-language information retrieval"

Komponen yang dibuat sendiri:
1. Wordnet (Sinonim)
2. NLG (buat ngerangkai kalimat)

# Open Dataset

In [158]:
import os
import json

path = './dataset'
dataset = []
 
directories = os.listdir(path)
for dir_name in directories:
    path_file = path + '/' + dir_name
    files = os.listdir(path_file)
    for file_name in files:
        path_to_open = path_file + '/' + file_name
        with open(path_to_open, 'r') as f:
            tweet_info = {}
            read_file = json.loads(f.read())
            tweet_info['trending_topic'] = dir_name
            tweet_info['tweet'] = read_file['text']
            dataset.append(tweet_info)

In [159]:
def save_dataset(dataset):
    dataset.to_csv('twitter_data.csv', encoding='utf-8')

In [160]:
import pandas as pd

df_tweets = pd.DataFrame(dataset)
df_tweets_train = df_tweets.copy()
df_tweets.shape

(1857, 2)

In [161]:
df_tweets['trending_topic'].value_counts()

#JanganSuriahkanIndonesia    1857
Name: trending_topic, dtype: int64

In [162]:
df_tweets['tweet'].iloc[4]

'# JanganSuriahkanIndonesia'

# Pre-processing
https://github.com/riochr17/Named-Entity-Tagger-ID
<br>
https://github.com/lilia-simeonova/preprocessing-tweets
<br>
http://iopscience.iop.org/article/10.1088/1742-6596/801/1/012072/pdf
<br>
http://taufiksutanto.blogspot.com/2018/01/tokenization-dalam-bahasa-inggris.html
<br>
http://www.yasirblog.com/2017/05/normalisasi-data-text-text.html
<br>
http://norvig.com/spell-correct.html
<br>
https://github.com/sastrawi/nlp-bahasa-indonesia

In [163]:
from py_translator import Translator
import enchant

def translate(text):
    counter = 1;
    checker = enchant.Dict("en_US")
    split_text = text.split(' ')
    words = []
    translator = Translator()
    for word in split_text:
        if word != '':
            if checker.check(word) and word != 'twitter':
                translated_word = translator.translate(text=word, dest='id', src='en').text 
                words.append(translated_word)
                if counter > 10:
                    delay_request(5,7)
                    counter = 1
            else:
                words.append(word.replace(' ',''))
    return " ".join(words)

In [164]:
from time import sleep
import random

def delay_request(lower, upper):
    print(random.uniform(lower, upper))
    sleep(random.uniform(lower, upper))

In [165]:
import re

def remove_url(text):
    return re.sub(r'http\S+', '', text)

In [166]:
def convert_text_to_lowercase(text):
    return text.lower()

In [167]:
def remove_emoji(text):
    return re.sub(r'(^|\s)(:D|:\/|:\)+|;\)|:-\))(?=\s|[^[:alnum:]+-]|$)', '', text)

In [168]:
import json

def spell_checker(text):
    words = text.split(' ')
    with open('spellchecker.json') as file_opened:
        text = file_opened.read()
    word_checker = json.loads(text)
    for idx in range(len(words)):
        if words[idx] in word_checker:
            words[idx] = word_checker[words[idx]]
    return " ".join(words)

In [169]:
def substitue_character(text):
    new_text = text.encode('ascii',errors='ignore').decode('utf-8', 'ignore')
    new_text = new_text.replace('\n', ' ')
    new_text = new_text.replace('?', ' ')
    new_text = new_text.replace('"', '')
    new_text = new_text.replace("'", '')
    new_text = re.sub(r'[.,\/!$%\^&\*;:{}=\`~()]', ' ', new_text)
    return new_text

In [170]:
def dwilingga(text):
    match = re.match(r'\w*(?=2(nya)?)', text)
    if match is not None:
        words = text.split(' ')
        for idx in range(len(words)):
            match_word = re.match(r'\w*(?=2(nya)?)', words[idx])
            if match_word is not None:
                words[idx] = words[idx].replace('2', '-' + match_word.group())
        return " ".join(words)
    return text

In [171]:
def remove_code_word(text):
    match = re.search(r'(?:[\d]+[\w]|[\w]+[\d])[\w\d]*', text)
    if match is not None:
        words = text.split(' ')
        for idx in range(len(words)):
            match_word = re.match(r'(?:[\d]+[\w]|[\w]+[\d])[\w\d]*', words[idx])
            match_hastag = re.match(r'(#[\w\d]*)', words[idx])
            if match_word is not None and match_hastag is None:
                words[idx] = ''
        return " ".join(words)
    return text

In [172]:
def prefix_fixing(text):
    match = re.search(r'\b[bkmpst][bcdfghjklmnpqrstvwxyz]', text)
    words = text.split(' ')
    if match is not None:
        for idx in range(len(words)):
            match_word = re.search(r'\b[bkmpst][bcdfghjklmnpqrstvwxyz]', words[idx])
            if match_word is not None:
                words[idx] = words[idx][0] + 'e' + words[idx][1:]
    match3 = re.search(r'\b[p][bcdefghjklmnpqstvwxyz]', text)
    if match3 is not None:
        for idx in range(len(words)):
            match_word = re.search(r'\b[p][bcdfghjklmnpqstvwxyz]', words[idx])
            if match_word is not None:
                words[idx] = words[idx][0] + 'i' + words[idx][1:]
    match2 = re.search(r'\b[d][bcdefghjklmnpqrstvwxyz]', text)
    if match2 is not None:
        for idx in range(len(words)):
            match_word = re.search(r'\b[d][bcdfghjklmnpqrstvwxyz]', words[idx])
            if match_word is not None:
                words[idx] = words[idx][0] + 'i' + words[idx][1:]
    return " ".join(words) if match is not None or match2 is not None or match3 is not None else text

In [173]:
def rank_word(text):
    match = re.match(r'(?!ke)\d*\b', text)
    number_dict = {
        '1': 'satu',
        '2': 'dua',
        '3': 'tiga',
        '4': 'empat',
        '5': 'lima',
        '6': 'enam',
        '7': 'tujuh',
        '8': 'delapan',
        '9': 'sembilan'
    }
    if match is not None:
        words = text.split(' ')
        for idx in range(len(words)):
            match_word = re.match(r'(?!ke)\d*\b', words[idx])
            if match_word is not None:
                words[idx] = words[idx].replace('2', '-' + match_word.group())
        return " ".join(words)
    return text

In [174]:
def remove_mention(text):
    return re.sub(r'@[\w\d_]+', '', text)

In [175]:
def cut_to_many_character(text):
    list_of_char = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    list_of_vocal = ['a', 'i', 'u', 'e', 'o']
    new_text = text
    for idx in range(len(list_of_vocal)):
        for jdx in range(idx, len(list_of_vocal)):
            pattern = r'\b' + list_of_vocal[idx] + list_of_vocal[jdx] + r'\b'
            pattern_2 = r'\b' + list_of_vocal[jdx] + list_of_vocal[idx] + r'\b'
            pattern_3 = r'\b[bcdfghjklmnpqrstvwxyz][bcdfghjklmnpqrstvwxyz]\b'
            new_text = re.sub(pattern, '', new_text)
            new_text = re.sub(pattern_2, '', new_text)
            new_text = re.sub(pattern_3, '', new_text)
    for character in list_of_char:
        pattern = character + r'{3,}'
        pattern_2 = character + r'{2,}\b'
        new_text = re.sub(pattern, character, new_text)
        new_text = re.sub(pattern_2, character, new_text)
    new_text = re.sub(r'(wk){2,}', '', new_text)
    new_text = re.sub(r'(ha){2,}', '', new_text)
    new_text = re.sub(r'(he){2,}', '', new_text)
    new_text = re.sub(r'(hi){2,}', '', new_text)
    new_text = re.sub(r'(bla){2,}', '', new_text)
    new_text = re.sub(r'\b[a-z]\b', '', new_text)
    new_text = re.sub(r'\b(kw)\b', '', new_text)
    new_text = re.sub(r'\b(ha)\b', '', new_text)
    new_text = re.sub(r'\b(he)\b', '', new_text)
    new_text = re.sub(r'\b(bla)\b', '', new_text)
    new_text = re.sub(r'\b(hi)\b', '', new_text)
    new_text = re.sub(r'\b(co)\b', '', new_text)
    new_text = re.sub(r'\b(id)\b', '', new_text)
    new_text = re.sub(r'\b(com)\b', '', new_text)
    new_text = re.sub(r'\b(org)\b', '', new_text)
    new_text = re.sub(r'\b(net)\b', '', new_text)
    new_text = re.sub(r'\b(gov)\b', '', new_text)
    new_text = remove_mention(new_text)
    new_text = remove_code_word(new_text)
    new_text = dwilingga(new_text)
    new_text = rank_word(new_text)
    new_text = prefix_fixing(new_text)
    new_text = re.sub(r' {2,}', ' ', new_text)
    return new_text

In [176]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

def remove_stop_words(sentence):
    factory = StopWordRemoverFactory().create_stop_word_remover()
    return factory.remove(sentence)

In [177]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(sentence):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(sentence)

In [178]:
def preprocess_text(text):
    new_text = substitue_character(text)
    new_text = convert_text_to_lowercase(new_text)
    new_text = remove_url(new_text)
    new_text = remove_emoji(new_text)
    new_text = cut_to_many_character(new_text)
#     new_text = translate(new_text)
    new_text = remove_stop_words(new_text)
    new_text = spell_checker(new_text)
    new_text = stemming(new_text)
    new_text = new_text.replace('#', '')
    return new_text

In [179]:
df_tweets_train['tweet'] = df_tweets_train['tweet'].apply(preprocess_text)

In [180]:
df_tweets_train['tweet'].iloc[456]

'jangansuriahkanindonesia moga indonesia aman umat islam moga satu adu domba nama apa sesal negri beranta jangansuriahkanindonesia'

In [181]:
df_tweets['tweet'].iloc[456]

'# JanganSuriahkanIndonesia  ya! Semoga Indonesia tetap aman, ummat Islam semoga tetap bersatu, jangan mau diadu domba atas nama apapun !!! Jangan sampai nanti kita menyesal setelah negri kita berantakan #JanganSuriahkanIndonesia'

In [182]:
dummy = []
for text in df_tweets_train['tweet']:
    split_text = text.split(' ')
    for word in split_text:
        dummy.append(word)
unique = set(dummy)
text_list = {}
for word in unique:
    text_list[word] =''

In [183]:
import json

with open('uniquejson.json', '+w') as opened_file:
    json.dump(text_list, opened_file, indent=4)

In [184]:
def word_frequencies_list(tweets):
    word_frequencies = []
    word_dict = []
    for tweet in tweets:
        words_in_sentence = tweet.split(' ')
        for word in words_in_sentence:
            if word in word_dict:
                word_frequencies[word_dict.index(word)] += 1
            else:
                word_frequencies.append(1)
                word_dict.append(word)
    return word_dict, word_frequencies

In [185]:
def reduce_word_list(word_dict, word_frequencies, treshold=2):
    new_word_frequencies = word_frequencies.copy()
    new_word_dict = word_dict.copy()
    idx = 0;
    while idx < len(new_word_frequencies):
        if new_word_frequencies[idx] < treshold:
            new_word_dict.pop(idx)
            new_word_frequencies.pop(idx)
        else:
            idx += 1
    return new_word_dict, new_word_frequencies

In [186]:
def graph_frequencies_list_all(tweets):
    graph = []
    graph_idx = 0
    for tweet in tweets:
        words_in_sentence = tweet.split(' ')
        prev_idx = -1
        idx_now = 0
        for word in words_in_sentence:
            if word != '':
                is_not_in_graph = True
                for idx_graph in range(len(graph)):
                    if graph[idx_graph]['word'] == word:
                        graph[idx_graph]['counter'] += 1
                        is_not_in_graph = False
                        idx_graph_representation = idx_graph
                        break
                if is_not_in_graph:
                    node = {
                        'idx': graph_idx,
                        'word': word,
                        'counter': 1,
                        'next_neighbours': [],
                        'prev_neighbours': []
                    }
                    if prev_idx != -1:
                        node['prev_neighbours'].append(prev_idx)
                        if graph_idx not in graph[prev_idx]['next_neighbours'] and graph_idx != prev_idx:
                            graph[prev_idx]['next_neighbours'].append(graph_idx)
                    prev_idx = graph_idx
                    graph_idx += 1
                    graph.append(node)
                else:
                    if prev_idx != -1:
                        if prev_idx not in graph[idx_graph_representation]['prev_neighbours'] and idx_graph_representation != prev_idx:
                            graph[idx_graph_representation]['prev_neighbours'].append(prev_idx)
                        if idx_graph_representation not in graph[prev_idx]['next_neighbours'] and idx_graph_representation != prev_idx:
                            graph[prev_idx]['next_neighbours'].append(idx_graph_representation)
                    prev_idx = idx_graph_representation
    return graph

In [442]:
def graph_frequencies_list_reduce(tweets, treshold=2):
    words, freqs = word_frequencies_list(tweets)
    new_words, new_freqs = reduce_word_list(words, freqs, treshold)
    graph = []
    graph_idx = 0
    for tweet in tweets:
        words_in_sentence = tweet.split(' ')
        prev_idx = -1
        idx_now = 0
        for word in words_in_sentence:
            if word != '' and word in new_words:
                is_not_in_graph = True
                for idx_graph in range(len(graph)):
                    if graph[idx_graph]['word'] == word:
                        graph[idx_graph]['counter'] += 1
                        is_not_in_graph = False
                        idx_graph_representation = idx_graph
                        break
                if is_not_in_graph:
                    node = {
                        'idx': graph_idx,
                        'word': word,
                        'counter': 1,
                        'next_neighbours': [],
                        'prev_neighbours': []
                    }
                    if prev_idx != -1:
                        node['prev_neighbours'].append(prev_idx)
                        if graph_idx not in graph[prev_idx]['next_neighbours'] and graph_idx != prev_idx:
                            graph[prev_idx]['next_neighbours'].append(graph_idx)
                    prev_idx = graph_idx
                    graph_idx += 1
                    graph.append(node)
                else:
                    if prev_idx != -1:
                        if prev_idx not in graph[idx_graph_representation]['prev_neighbours'] and idx_graph_representation != prev_idx:
                            graph[idx_graph_representation]['prev_neighbours'].append(prev_idx)
                        if idx_graph_representation not in graph[prev_idx]['next_neighbours'] and idx_graph_representation != prev_idx:
                            graph[prev_idx]['next_neighbours'].append(idx_graph_representation)
                    prev_idx = idx_graph_representation
    return graph

In [443]:
import math

def calculate_weight(counter, distance, base):
    return counter - (distance * (math.log(counter)/math.log(base)))

In [444]:
def get_next_path(idx, graph, distance, base, data={'path': [], 'weight': 0}, iterator=5):
    if len(graph[idx]['next_neighbours']) == 0 or iterator == distance:
        new_data = {
            'path': data['path'].copy(),
            'weight': data['weight']
        }
        if idx not in data['path']:
            new_data['path'].append(idx)
            new_data['weight'] += calculate_weight(graph[idx]['counter'], distance, base)
        return new_data
    else:
        new_data = {
            'path': data['path'].copy(),
            'weight': data['weight']
        }
        if idx not in data['path']:
            new_data['path'].append(idx)
            new_data['weight'] += calculate_weight(graph[idx]['counter'], distance, base)
            neighbours = []
            for next_idx in graph[idx]['next_neighbours']:
                data_send = {
                    'path': new_data['path'].copy(),
                    'weight': data['weight']
                }
                if next_idx not in data_send['path']:
                    neighbours.append(get_next_path(next_idx, graph, distance + 1, base, data_send, iterator))
            data_max = {'path': [], 'weight': 0}
            for neighbour in neighbours:
                if neighbour['weight'] > data_max['weight']:
                    data_max = neighbour.copy()
            return data_max.copy()
        else:
            return new_data.copy()

In [445]:
def get_prev_path(idx, graph, distance, base, data={'path': [], 'weight': 0}, iterator=5):
    if len(graph[idx]['next_neighbours']) == 0 or iterator == distance:
        new_data = {
            'path': data['path'].copy(),
            'weight': data['weight']
        }
        if idx not in data['path']:
            new_data['path'].insert(0, idx)
            new_data['weight'] += calculate_weight(graph[idx]['counter'], distance, base)
        return new_data
    else:
        new_data = {
            'path': data['path'].copy(),
            'weight': data['weight']
        }
        if idx not in data['path']:
            new_data['path'].insert(0, idx)
            new_data['weight'] += calculate_weight(graph[idx]['counter'], distance, base)
            neighbours = []
            for next_idx in graph[idx]['prev_neighbours']:
                data_send = {
                    'path': new_data['path'].copy(),
                    'weight': data['weight']
                }
                if next_idx not in data_send['path']:
                    neighbours.append(get_prev_path(next_idx, graph, distance + 1, base, data_send, iterator))
            data_max = {'path': [], 'weight': 0}
            for neighbour in neighbours:
                if neighbour['weight'] > data_max['weight']:
                    data_max = neighbour.copy()
            return data_max.copy()
        else:
            return new_data.copy()

In [467]:
def get_topic_index(graph, topic):
    for node in graph:
        if node['word'] == topic:
            return node['idx']
    return -1

In [468]:
def combine_path(path_next, path_prev):
    all_path = {
        'path': path_prev['path'][:-1] + path_next['path'],
        'weight': path_prev['weight'] + path_next['weight']
    }
    return all_path

In [469]:
def get_sentence(graph, path):
    sentence = ''
    for item in path['path']:
        for node in graph:
            if node['idx'] == item:
                sentence += node['word'] + ' '
    return sentence

In [502]:
def summarize(tweets, topic, base=2, iterator=5, treshold=10):
    graph = graph_frequencies_list_reduce(tweets, treshold)
    topic_index = get_topic_index(graph, topic)
    if topic_index == -1:
        return 'this string cannot be a topic'
    path_next = get_next_path(topic_index, graph, 0, base, iterator=iterator)
    path_prev = get_prev_path(topic_index, graph, 0, base, iterator=iterator)
    all_path = combine_path(path_next, path_prev)
    return get_sentence(graph, all_path)

In [521]:
trendinc_topic = df_tweets_train['trending_topic'].iloc[0].replace('#', '').lower()
sentence = summarize(df_tweets_train['tweet'], trendinc_topic)

In [525]:
print(sentence)

twitter muslim negara kehilafah indonesia jangansuriahkanindonesia hoax alhamdulillah ga bangsa twitter 
